<a href="https://colab.research.google.com/github/Davidfdaf/Optimization-modeling-course/blob/main/showing_off_bayesian_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qpsolvers

     |████████████████████████████████| 121 kB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for quadprog: filename=quadprog-0.1.11-cp37-cp37m-linux_x86_64.whl size=290749 sha256=8585367118a297462e3dec2854df0e90d6491f4b768a41a4eff4e4e61e0cd5a2
  Stored in directory: /root/.cache/pip/wheels/4a/4e/d7/41034ea11aeef1266df3cae546116cb6094e955c41ae3e2589
Successfully built quadprog


In [2]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 6.4 MB 11.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import yfinance as yfin
yfin.pdr_override()
import qpsolvers
from sklearn.gaussian_process import GaussianProcessRegressor
from numpy.random import uniform
from scipy.stats import norm
import time
from pandas.core.common import flatten

#Importing backtest and optimization functions

In [28]:
def import_data(tickers, start, end, give = "R", interval = "W"):#read in stock data  
    """
    tickers: tickers to read 
    start, end: dates in datetime.datetime(2015,12,1) format
    give: return prices or returns 
    timeframe: resample dataframe to be D:daily data, W:Weekly Data, M:Monthly data, A:Annual data
    """
    
    stocks = pd.DataFrame()
    stocks[tickers] = data.get_data_yahoo(tickers, start, end)['Adj Close']
        
    if(give == "R"):
        if (interval=="D"):
            return stocks.pct_change().dropna()
        else:
            return stocks.pct_change().dropna().resample(interval).apply(lambda x: ((x + 1).cumprod()-1).last("D"))
        
    if(give == "P"):
        return stocks
    
#importing financial data 
#commodity_futures = ['GC=F', 'SI=F', 'CL=F']
#cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']
#currencies = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
#mutual_funds = ['PRLAX', 'QASGX', 'HISFX']
#Interest rates = ['^TNX', '^IRX', '^TYX']




def mean_var_opt(posLB, posUB, t, tickers, increments_for_graph, Cov, muf, mean_vect):
    """
    posLB,posUB: upper and lower bounds for individual positions
    t: number to multiply expected returns by to make them yearly
    tickers: list of column names
    increments_for_graph: decreasing this number increases the number of portfolios calculated
    Cov: covariance of returns
    muf: should be a single value
    mean_vect: should be mean of dataframe

    """
    Cov = 2 * np.array(Cov)
    mean_vect = mean_vect * t
    muf  = muf * t
    m = len(tickers)

    from scipy.optimize import linprog
    #set equality constraints. I want my positions to sum to 1
    A_eq = np.array((1,)*m).reshape(1,-1)
    b_eq = [1]

    # set bounds on leverage. We can short but we can not go long
    bounds = np.vstack([(posLB,)*m, (posUB,)*m]).T

    #find min and max return
    min_rtn = mean_vect @ linprog(c=mean_vect, A_eq = A_eq, b_eq = b_eq, bounds=bounds).x
    max_rtn = mean_vect @ linprog(c=-mean_vect, A_eq = A_eq, b_eq = b_eq, bounds=bounds).x

    from qpsolvers import solve_qp
    muP = np.arange(min_rtn * .995,max_rtn * .995, increments_for_graph)
    sdP = np.zeros(len(muP))
    weights = np.zeros([len(muP),m])

    q_vec = np.zeros(m).reshape(-1,)
    G = np.zeros([m,m])
    h = np.zeros(m)
    A = np.vstack([np.array((1,)*m).reshape(1,-1),mean_vect])

    #calculate each optimized portfolio for each mean
    for i in range(len(muP)):
        b = np.array([1,muP[i]])
        weights[i] = solve_qp(P=Cov, q = q_vec, G=G, h=h, A=A, b=b, lb = bounds[:,0], ub = bounds[:,1])
        sdP[i] = np.sqrt(weights[i] @ (Cov/2) @ weights[i]) * np.sqrt(t)
    
    portfolios = pd.DataFrame({'Returns': muP, 'Volatility': sdP,})
    for counter, symbol in enumerate(tickers):
        portfolios[symbol+' weight'] = [w[counter] for w in weights]
    
    portfolios['Sharpe'] = (portfolios['Returns']-muf) / portfolios['Volatility'] #note using monthly returns and vol

    tangent = portfolios.iloc[[portfolios.Sharpe.argmax()]]
    min_var = portfolios.iloc[[portfolios.Volatility.argmin()]]
    efficient_portfolios = portfolios[portfolios['Returns'] >= min_var.Returns.values[0]-.02] #the .02 is arbitrary to see a bit below the min var port too

    return tangent, min_var, efficient_portfolios

###Bayesian optimization

In [29]:
# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat = model.predict(X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = model.predict(Xsamples, return_std=True)
    mu = mu.reshape(-1,)
    std = std.reshape(-1,)
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model,search_samp):
    # random search, generate random samples
    Xsamples = constrained_samp(search_samp)
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmax(scores)
    return Xsamples[ix]

def bayes_opt(starting_data = 50, search_samp = 100, optimization_steps = 50):
    """
    startig data: number of actual points to start with
    search samp: number of points to sample at each optimization step
    optimization_steps: number of updates
    """
    
    t0 = time.time()

    #samples
    X = constrained_samp(starting_data)
    ys = np.asarray([func_approx(x) for x in X]).reshape(-1,1) ####
    # define the model
    model = GaussianProcessRegressor()
    model.fit(X, ys)

    # perform the optimization process
    error=[]
    for i in range(optimization_steps):
        # select the next point to sample
        point = opt_acquisition(X, ys, model, search_samp)  #####
        # sample the point
        actual = func_approx(point) ####
        # summarize the finding
        est = model.predict(point.reshape(1,-1))
        wrongness=abs(actual-est)/(est+.0001)
        error.append(wrongness.item())
        # add the data to the dataset
        X = np.vstack((X, point))
        ys = np.vstack((ys, [[actual]]))
        # update the model
        model.fit(X, ys)
    
    # best result
    X = np.round(X,2)
    ix = np.argmax(ys)
    print('Best Result: \nWeights=%s, \n function max=%.6f' % (tuple(X[ix]), ys[ix]))
    print('\nMean Error=%.3f' % (np.mean(error)))
    t1 = time.time()
    print("time",t1-t0)
    return(X,ys,ix,error)

###Backtesting function

In [30]:
def backtest(weights, col, dataf, days_between_rebalance, rebalance_func, show_weights=False, wealth = 1, RebalanceOffset=0, give='R',formation = 0): #cols allows users to backtest subsets of dataframe
    """
    weights: tells the function the number of assets. sets the starting weights and is the weights for constant rebalancing
    wealth: sets starting value
    RebalanceOffset: to offset day of rebalances by these values to provide information about rebalance timing luck
    give: return returns or prices
    formation: number of days covariance matrix is formed over
    
    """
    colin=[0]*len(weights)
    port=[0]*len(weights)
    worth=[wealth]

    for n in range(len(weights)):
        colin[n] = dataf.columns.get_loc(col[n])
        port[n]=wealth*weights[n]

    for n in range (formation,len(dataf)):
        port = np.multiply(port, np.array((1+dataf.iloc[n,colin])))

        end_of_day = sum(port)
        worth.append(end_of_day)
        
        if ((n + RebalanceOffset)% days_between_rebalance==0):        #rebalance
            port=rebalance_func(dataf, n, formation, port, weights, sum(port))
            if show_weights == True:
                print(np.round(np.array(port/sum(port)),2))
                
    if (give=='R'):            
        p_rtn = pd.DataFrame(worth).pct_change()[1:].set_index(dataf.index[formation:])
        return(p_rtn)
    if (give=='P'):
        return(pd.DataFrame(worth[1:]).set_index(dataf.index[formation:]))
    
def constant_weight_rebalance(stocks, n, formation_period, port, weights, total):  
    update = np.subtract(np.array(weights), np.array(port/total))/1
    new_port = np.add(np.array(port/total),update)    #update portfolio weights
    #print(np.round(new_port,2),n)                       #print weights
    return total * np.array(new_port)

def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp

def ivp_rebalance(stocks, n, formation_period, port, weights, total):
    cov = stocks[n-formation_period : n - 1].cov() 
    ivp = list(flatten(getIVP(cov).reshape(-1,1)))
    update = np.subtract(np.array(ivp), np.array(port/total))/1
    #print(abs(update).sum()/2)                       #print turnover
    new_port = np.add(np.array(port/total),update)    #update portfolio weights
    return total * np.array(ivp)

In [67]:
#import data
start = datetime(2002,1,1)
end = datetime(2012,1,16)
assets = ['TLT','SPY']
df = import_data(assets,start,end, give ="R",interval = "D")

[*********************100%***********************]  2 of 2 completed


In [68]:
df.head()

,TLT,SPY
Date,,
2002-07-31,0.002419,0.012389
2002-08-01,-0.026108,0.005696
2002-08-02,-0.022415,0.010241
2002-08-05,-0.034797,0.004412
2002-08-06,0.033664,-0.008550


In [69]:
#form mean variance optimized ports and return tangent port and minimum variance ports
tangent, min_var, efficient = mean_var_opt(posLB = 0, posUB = 1, t=252, tickers=assets, increments_for_graph=.001, \
             Cov = np.array(df.cov()), muf = 0, mean_vect = df.mean())

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [70]:
tangent #note this is only the return and volatility according to the statistical relationships. not an actual backtested performance

,Returns,Volatility,TLT weight,SPY weight,Sharpe
11,0.090754,0.091174,0.331809,0.668191,0.995397


###Using bayesian optimization

In [71]:
#To use the bayes optimization algo you need to define the function you want to output(it must output a single value), 
#and the input space you sample from

#note: dont change the names of the functions as they are called in the alogrithm above
#note: bayes opt does better if function being evaluated is in the range of 1-1000
#note: if mean error is high increase initial sample size

In [74]:
def func_approx(x):
    perform = backtest(x, df.columns, df, days_between_rebalance = 63, rebalance_func = constant_weight_rebalance)
    return (perform.mean()*252 / (perform.std()*np.sqrt(252))).values[0] #sharpe proxy

N = len(df.columns)
def constrained_samp(M):
    H=np.zeros([M,N+1])
    U=np.zeros([M,N])
    for j in range (0,M):
        for i in range(1,N): 
            H[j,i] = round(uniform(0,1),2)
        H[j,N] = 1
        H.sort()
        for i in range(1,N+1):
            U[j,i-1] = H[j,i] - H[j,i-1]
    return(U)

In [87]:
weights, outputs, best, error = bayes_opt(starting_data = 15, search_samp = 200, optimization_steps = 5)

Best Result: 
Weights=(0.35, 0.65), 
 function max=0.949551

Mean Error=0.003
time 28.239130973815918


In [86]:
#actual backtest of mean variance optimization method for comparison
func_approx(tangent.iloc[:,2:4].values[0])

0.9480490034850723

In [ ]:
#you can see the bayesian optimization algorithm finds the more optimal solution so it works, but the important thing is it's flexability
#you can change func_approx to output any metric about the portfolio returns you want to maximize